<h1 align='center'> Introduction to Python Decorators </h1>

<img src="./images/flask decorator.png" alt="decorator used in flask">

<img src="./images/standard_lib_decorator.png" alt="stamdard lib decorator">

<h2>Decorators are such an important part of Python </h2>

<img src="./images/pep318.png" alt="pep318, decorators for functions and methods">

<img src="./images/pep3129.png" alt="pep3129, class decorators">

### What are Decorators?

> Decorators "decorate" or "wrap" another function and let you execute code before and after the wrapped function runs.

> A callable that returns another callable.

### Why Decorators?


* Decorators allow you to extend and modify the behavior of a callable (functions, methods, and classes)<br></br><br></br>
* Allow you to do so without modifying the callable itself (the callable's behavior changes only when it's decorated)

### Use Case for decorators 
* logging
* enforcing access control and authentication
* instrumentation and timing functions
* rate-limiting
* caching, 

e.g. `%timeit` in Ipython is a decorator under the hood!

### But first, a few things to remember...<br></br>

* Everything in Python is an object...including functions, classes, etc <br></br><br></br>
* Python functions are first-class objects

- **Demonstrate this on the chalkboard:** Names that we define are simply identifiers that point to objects. That means that various names can be bound to the same object

<img src="./images/obj_ref_meme.png" alt="re-assign variables meme">

### What does it mean for functions to be first-class objects?<br></br>

* They can be assigned to variables, passed to, and returned from other functions<br></br><br></br>
* Functions can be defined inside other functions

In [1]:
# a function can be assigned to a variable
def print_greet():
    return "Hello World!"

say_it = print_greet
say_it()

'Hello World!'

In [2]:
# a function can return another function
def is_called():
    def is_returned():
        print("Hello world!")
    return is_returned

new = is_called()

new()

Hello world!


In [3]:
# Functions can be passed to other functions

def mind_blowing(func):
    check = func("I hope this is not too difficult to grok")
    print(check)
    
def is_question(text):
    return text + '?'

mind_blowing(is_question)

I hope this is not too difficult to grok?


* Functions that accept other functions as arguments are called higher-order functions<br></br><br></br>

* Examples include map, filter, and reduce

<h3> Now back to decorators </h3>

#### Let's start with a simple example

In [4]:
def null_decorator(func):
    return func

def greet():
    return "Hello PyEdmonton"

null_greeting = null_decorator(greet)
null_greeting()

'Hello PyEdmonton'

<h3> We could, of course, add some (syntatic) sugar to this: </h3>

In [5]:
@null_decorator
def greet():
    return "Hello again!"

greet()

'Hello again!'

This is not a very useful implementation of course, but hopefully has set the tone for what decorators actually do.

The con of using the @ syntax is that it becomes difficult to access the undecorated function, so the manual form might be useful sometimes.

<h3> A more useful example...</h3>

In [6]:
def uppercase(func):
    def wrapper():
        original = func()
        modified = original.upper
        return modified
    return wrapper

@uppercase
def confirm_understanding():
    return "Ok so far?"

confirm_understanding()

<function str.upper()>

* The wrapper closure has access to the undecorating input function<br></br><br></br>

* The wrapper can execute additional code before and after calling the input function

In [7]:
def explicit_titlecase(func):
    def wrapper():
        print("About to decorate the function")
        print('------------------------------')
        modified = func().title()
        print("Finished decorating the function; output should be all title case")
        return modified
    return wrapper

@explicit_titlecase
def confirm_understanding():
    return "Still good?"

confirm_understanding()

About to decorate the function
------------------------------
Finished decorating the function; output should be all title case


'Still Good?'

<h3> So, this is what we know so far...</h3><br></br>

* Decorators modify the behavior of a callable through a wrapper closure<br></br><br></br>

* You don't have to permanently modify the original callable to change it's behavior<br></br><br></br>

* The behavior of the original callable only changes when it is decorated<br></br><br></br>

In [8]:
def explicit_titlecase(func):
    def wrapper():
        print("About to decorate the function")
        print('------------------------------')
        modified = func().title()
        print("Finished decorating the function; output should be all title case")
        return modified
    return wrapper

def confirm_understanding():
    return "Still good?"

# can use original callable without the decorator
confirm_understanding()

'Still good?'

In [9]:
# or can decorate the callable to modify it's behavior
explicit_titlecase(confirm_understanding)()

About to decorate the function
------------------------------
Finished decorating the function; output should be all title case


'Still Good?'

<h3> Applying Multiple Decorators to a Function </h3>

In [10]:
def div(func):
    def wrapper():
        return '<div>' + func() + '</div>'
    return wrapper

def paragraph(func):
    def wrapper():
        return '<p>' + func() + '</p>'
    return wrapper

In [11]:
@div
@paragraph
def greet_again():
    return "Hello again!"

In [12]:
greet_again()

'<div><p>Hello again!</p></div>'

In [13]:
@paragraph
@div
@paragraph
def no_more_greet():
    return "This is the last time I'm using greet, I swear!"

In [14]:
no_more_greet()

"<p><div><p>This is the last time I'm using greet, I swear!</p></div></p>"

<h4> When you apply multiple decorators to a function </h4>

* The decorators are applied from bottom to top<br></br><br></br>

* Much like building up a stack<br></br><br></br>

* A really deep level of decorator stacking might eventually affect performance

<h3> Decorating Functions That Accept Arguments </h3>

> \*args and \**kwargs to the rescue!

In [15]:
def proxy(func):
    def wrapper(*args, **kwargs):
        return func(*args, **kwargs)
    return wrapper

* \* and \** operators in the wrapper closure definition to collection all positional and keyword arguments<br></br><br></br>
* Wrapper closure forwards the collected arguments to the original function

<h3> Now, time for a practical example </h3>

* Let's write a trace (logging) decorator that logs function arguments and results during execution

In [16]:
def trace(func):
    def wrapper(*args, **kwargs):
        print(f'TRACE: calling {func.__name__}() '
             f'with {args}, {kwargs}')
        
        original = func(*args, **kwargs)
        
        print(f'TRACE: {func.__name__}() '
             f'returned: {original!r}')
        return original
    return wrapper

In [17]:
@trace
def whodunnit(name, thing, venue, feel):
    return f"{name} {thing} {venue} {feel}"

In [18]:
whodunnit("I", "presented on decorators at", venue="PyEdmonton", feel="And it felt great!")

TRACE: calling whodunnit() with ('I', 'presented on decorators at'), {'venue': 'PyEdmonton', 'feel': 'And it felt great!'}
TRACE: whodunnit() returned: 'I presented on decorators at PyEdmonton And it felt great!'


'I presented on decorators at PyEdmonton And it felt great!'

<h3>  Debugging Decorated Functions </h3><br></br>

* One downside of decorating functions is that it "hides" some of the metadata of the original undecorated function


In [19]:
def whodunnit(name, thing, venue, feel):
    """Docstring goes here"""
    return f"{name} {thing} {venue} {feel}"

decorated_whodunnit = trace(whodunnit)

In [20]:
# metadata for original function is transparent to user
print(whodunnit.__name__)
print(whodunnit.__doc__)

whodunnit
Docstring goes here


In [21]:
# but not so when the function is decorated
print(decorated_whodunnit.__name__)
print(decorated_whodunnit.__doc__)

wrapper
None


<h4> But do not despair, there is a way...</h4>

In [22]:
from functools import wraps

def trace(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        print(f'TRACE: calling {func.__name__}() '
             f'with {args}, {kwargs}')
        
        original = func(*args, **kwargs)
        
        print(f'TRACE: {func.__name__}() '
             f'returned: {original!r}')
        return original
    return wrapper

In [23]:
@trace
def whodunnit(name, thing, venue, feel):
    """Docstring goes here"""
    return f"{name} {thing} {venue} {feel}"

In [24]:
print(whodunnit.__name__)
print(whodunnit.__doc__)

whodunnit
Docstring goes here


<h3> Wrap Up </h3>

* Knowledge of decorators are an important toolkit in your Python toolbox<br></br><br></br>

* Decorators help you modify the behavior of a callable without permanently changing the callable<br></br><br></br>

* Used extensively in the standard library and third party-modules<br></br><br></br>

* Ongoing proposal to relax grammar restrictions on decorators (PEP 614)<br></br><br></br>


<img src="./images/pep614.png" alt="pep for relaxing decorators grammar syntax">

<h3> References </h3>

1. Python Tricks - Dan Bader, Ch 3.3, The Power of Decorators <br></br><br></br>

2. Programiz Article on Decorators: https://www.programiz.com/python-programming/decorator <br></br><br></br>

3. Awesome Python Decorators: https://github.com/lord63/awesome-python-decorator<br></br><br></br>

4. Real Python Article on Decorators: https://realpython.com/primer-on-python-decorators/

<h3 align='center'> Slides available on Github </h3>

### https://github.com/ezebunandu/edmontonpy-talks